In [ ]:
import configparser
from sentence_transformers import SentenceTransformer
import torch
from PyPDF2 import PdfReader
import requests
from io import BytesIO
from pinecone import Pinecone

config = configparser.ConfigParser()
config.read('configuration.properties')

def pinecone_connection():
    try:
        pinecone_api_key = config['PINECONE']['pinecone_api_key']
        index_name = config['PINECONE']['index']
        return pinecone_api_key, index_name
    except Exception as e:
        print("Exception in pinecone_connection function: ",e)
        return "", ""

d:\Old Laptop\Hackathon Dev\Civic Hackathon\civic_ai\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def read_pdf_data(pdf_link):
    try:
        response = requests.get(pdf_link)
        if response.status_code == 200:
            pdf_bytes = BytesIO(response.content)

            # Step 2: Read and extract text
            reader = PdfReader(pdf_bytes)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"
        else:
            print("Failed to download the PDF.")
            text = ""
        return text
    except Exception as e:
        print("Exception in read_pdf_data function: ",e)
        return ""

In [5]:
pdf_link = 'https://public-inspection.federalregister.gov/2025-00479.pdf?1736516733'
read_pdf_data(pdf_link)

'Billing Code: 3410-30-P\nDEPARTMENT OF AGRICULTURE\nFood and Nutrition Service\nSummer Food Service Program; 2025 Reimbursement Rates\nAGENCY:  Food and Nutrition Service, USDA.\nACTION:  Notice.\nSUMMARY:  This notice informs the public of the annual adjustments to the \nreimbursement rates for meals served in the Summer Food Service Program. These \nadjustments address changes in the Consumer Price Index, as required under the Richard \nB. Russell National School Lunch Act. The 2025 reimbursement rates are presented as a \ncombined set of rates to highlight simplified cost accounting procedures. The 2025 rates \nare also presented individually, as separate operating and administrative rates of \nreimbursement, to show the effect of the Consumer Price Index adjustment on each rate. \nOn average, the 2025 rates adjustment represents a 3.6 percent increase in the rates from \nlast year. \nDATES:  This adjustment is applicable January 1, 2025.\nFOR FURTHER INFORMATION CONTACT:  Penny Bu